# notebook for troubleshooting and testing solutions for issues in the workflow

Note this is a testing notebook. In this notebook we test different approaches used in our workflow. Mostly these were tested locally, so the paths to data etc will note work.

In [ ]:
# tests to find solution for https://github.com/c-scale-community/use-case-hisea/issues/28

from datetime import timedelta, datetime
from pathlib import Path
import subprocess
import xarray as xr # note dependencies: dask, netCDF4

vars = ('thetao', 'bottomT', 'so', 'zos', 'uo', 'vo')
date_min = '2022-04-01'
date_max = '2022-04-05'

delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')

var = vars[0]
i = 0
day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
check_file = Path('/Users/backeb/Documents/data/tmp/cmems_'+str(var)+'_'+str(day)+'.nc')

j=0
while not check_file.is_file():
    print('j '+str(i))
    print(check_file.is_file())
    open(check_file, 'a')
    j+=1

print(check_file.is_file())
print(j)

In [ ]:
# alternate way to concat netcdf

from pathlib import Path
import xarray as xr

datasets = []
#for path in Path('/Users/backeb/Documents/data/tmp/').iterdir():
for path in sorted(Path('/Users/backeb/Documents/data/tmp/').rglob('*no3*.nc')):
    print(path)
    ds = xr.open_dataset(path, decode_times=False)
    datasets.append(ds)
combined = xr.concat(datasets, dim='time')

print(combined)

In [ ]:
# for download_era5.py get an error when iterating over months
# e.g. from 2021-07-01 to 2022-06-31 the monthstr is empty
# this is because I'm trying to do `for i in range(7, 6, 1): ...`
# tests to find a better way to iterate over a period

import pandas as pd
from datetime import datetime, timedelta
import numpy as np

date_min = '2021-07-01'
date_max = '2022-06-30'

yearstr = []
pdyears = pd.period_range(start=date_min, end=date_max, freq='Y')
[yearstr.append(f'{year:0>4}') for year in pdyears.year]
print(yearstr)

monthstr = []
pdmonths = pd.period_range(start=date_min, end=date_max, freq='M')
#print(np.unique(pdmonths.month))
[monthstr.append(f'{month:0>2}') for month in np.unique(pdmonths.month)]
print(monthstr)

daystr = []
pddays = pd.period_range(start=date_min, end=date_max, freq='D')
#print(np.unique(pddays.day))
[daystr.append(f'{day:0>2}') for day in np.unique(pddays.day)]
print(daystr)



In [ ]:
# tests to change download_era5.py to deal with CDS API limitation: https://github.com/c-scale-community/use-case-hisea/issues/30
from datetime import timedelta, datetime

date_min = '2021-07-01'
date_max = '2022-06-30'
vars = ('10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', '2m_dewpoint_temperature', 'relative_humidity', 'surface_net_solar_radiation', '2m_temperature', 'total_cloud_cover')

vars = list(vars)
print(vars)

delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')
i = 0
day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
yearstr = [f'{day.year:0>4}']
print(yearstr)
monthstr = [f'{day.month:0>2}']
print(monthstr)
daystr = [f'{day.day:0>2}']
print(daystr)



In [ ]:
# tests to figure out https://github.com/c-scale-community/use-case-hisea/issues/31
from datetime import timedelta, datetime
from pathlib import Path
import subprocess
import xarray as xr

username = 'insert'
password = 'insert'
longitude_min = 22.5
longitude_max = 24.5
latitude_min = 36.5
latitude_max = 38.5
date_min = '2022-04-01'
date_max = '2022-04-02'
vars = ('thetao', 'bottomT', 'so', 'zos', 'uo', 'vo')

#make the /data/tmp directory if it does not exist
Path('/data/tmp').mkdir(parents=True, exist_ok=True)
delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')
for var in vars:
        for i in range(delta.days+1):
                max_runs = 2
                run = 0
                day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
                check_file = Path('/data/tmp/cmems_'+str(var)+'_'+str(day)+'.nc')
                while not check_file.is_file() and run < max_runs:
                        try:
                                subprocess.run(['python', '-m', 'motuclient',
                                        '--motu', 'https://nrt.cmems-du.eu/motu-web/Motu',
                                        '--service-id', 'GLOBAL_ANALYSIS_FORECAST_PHY_001_024-TDS',
                                        '--product-id', 'global-analysis-forecast-phy-001-024',
                                        '--longitude-min',str(longitude_min),
                                        '--longitude-max', str(longitude_max),
                                        '--latitude-min', str(latitude_min),
                                        '--latitude-max', str(latitude_max),
                                        '--date-min', str(day)+' 12:00:00',
                                        '--date-max', str(day)+' 12:00:00',
                                        '--depth-min', '0.493',
                                        '--depth-max', '5727.918000000001',
                                        '--variable', str(var),
                                        '--out-dir', '/data/tmp',
                                        '--out-name', 'cmems_'+str(var)+'_'+str(day)+'.nc',
                                        '--user', username,
                                        '--pwd', password],
                                        check=True,
                                        timeout=300)
                        except subprocess.TimeoutExpired as e:
                                print(var)
                                print(e.stdout)
                                print(e.stderr)
                                continue
                        else:
                                break
                        finally:
                                run += 1
        ds = xr.open_mfdataset('/data/tmp/cmems_'+var+'_*.nc')
        ds.to_netcdf('/data/cmems_'+var+'.nc')

# helpers for snakemake `rule configure_workflow:`

In [24]:
import re
from datetime import datetime, timedelta


def get_date_range(run_mode, forecast_window_mid_pt=None, forecast_window=None, tstart=None, tstop=None, outfile=None):
    if run_mode == 'forecast':
        if forecast_window_mid_pt is None:
            raise ValueError('forecast_window_mid_pt must be defined in forecast mode')
        if not re.match(r'today - \d+ days', forecast_window_mid_pt):
            raise ValueError("forecast_window_mid_pt must have the format 'today - X days'")

        if forecast_window is None:
            raise ValueError('forecast_window must be defined in forecast mode')
        if not re.match(r'\d+ days', forecast_window):
            raise ValueError("forecast_window must have the format 'X days'")

        # Split the input string `forecast_window_mid_pt`
        parts = forecast_window_mid_pt.split(' - ')
        if len(parts) == 2 or parts[1][-4:] == 'days':
            # Parse the number of days to subtract
            days2subtract = int(parts[1][:-5])
        elif len(parts) == 1:
            days2subtract = 0

        # Extract the number from the string `forecast_window` using regular expressions
        days2forecast = int(re.findall(r'\d+', forecast_window)[0])

        # Calculate the date
        today = datetime.today().date()
        date_min = (today - timedelta(days=days2subtract)).strftime("%Y-%m-%d")
        date_max = (today - timedelta(days=days2subtract) + timedelta(days=days2forecast)).strftime("%Y-%m-%d")

    elif run_mode == 'hindcast':
        if tstart is None:
            raise ValueError('tstart must be defined in hindcast mode')
        if tstop is None:
            raise ValueError('tstop must be defined in hindcast mode')

        date_min = tstart
        date_max = tstop
    else:
        raise ValueError("run_mode must be either 'forecast' or 'hindcast'")

    # Open the file in read mode and read its contents
    with open(outfile, 'r') as f:
        file_contents = f.readlines()

    # Update the values of date_min and date_max in the file contents
    for i, line in enumerate(file_contents):
        if line.startswith('date_min:'):
            file_contents[i] = f'date_min: {date_min}\n'
        elif line.startswith('date_max:'):
            file_contents[i] = f'date_max: {date_max}\n'

    # Write the updated contents back to the file
    with open(outfile, 'w') as f:
        f.writelines(file_contents)

    return date_min, date_max

date_min, date_max = get_date_range(run_mode='forecast', forecast_window_mid_pt='today - 15 days', forecast_window='10 days', outfile='config.yml')

In [25]:
!cat config.yml

######################### DO NOT EDIT ANYTHING BELOW THIS LINE ###############################

# update date_min, date_max for each execution of the workflow
date_min: 2023-03-08
date_max: 2023-03-18
